# Check the three resource supplies

Need to find the simplest way of sampling, and what need to be changed in different codes to be compatible with different resource supplies. 

## Framework

The species and resources dynamics is given by 
$$
\frac{\mathrm{d} S_i}{\mathrm{d} t} = S_i \Big(\sum_{\alpha}G_{i\alpha}R_{\alpha} - \delta_i \Big),\\
\frac{\mathrm{d} R_{\alpha}}{\mathrm{d} t} = h_{\alpha}(R_\alpha) - R_{\alpha}\sum_{i}C_{i\alpha}S_i.
$$
Local Jacobian at fixed point is 
$$
   J^* = \left(\begin{array}{c|c}
       O  & D(S^*)G\\[1.0ex]
     \hline\\[-1.0ex]
     -D(R^*)C^\top   & \frac{\partial h}{\partial R} - D(C^\top S^*)
    \end{array}
    \right).
$$

## Constant supply

$h_\alpha = \gamma_\alpha$, such that $\frac{\partial h}{\partial R} = 0$.

## Linear supply

$h_{\alpha} = l_\alpha (R^{\bullet}_{\alpha} - R_{\alpha})$, where $l$ is the dilution rates, and $R^{\bullet}_{\alpha}$ is the source concentration.
$$\frac{\partial h}{\partial R} = -D(l).$$

## Logistic supply

$h_{\alpha} = g_{\alpha} R_{\alpha}(K_{\alpha} - R_{\alpha})$, where $g_{\alpha}K_{\alpha}$ is the logistic growth rate and $K_{\alpha}$ the carrying capacity.

$$
\frac{\partial h_\alpha}{\partial R_\alpha} = g_\alpha (K_{\alpha}-2R_\alpha) = \sum_i C_{i\alpha} S_i - g_\alpha R_\alpha
$$

## Calculate the real Jacobians

We can directly use these to draw figures. Sample and save for simulations.

In [1]:
# load the data G, C, S, R.
from scipy.io import loadmat # type: ignore
import torch

loadedGC = loadmat("./data/allsamples.mat")
C_span = torch.tensor(loadedGC['allC'])
G_span = torch.tensor(loadedGC['allG'])
S_span = torch.tensor(loadedGC['allSstar'])
R_span = torch.tensor(loadedGC['allRstar'])

Ns_span = torch.tensor(loadedGC['Ns_span']).squeeze(0)
Nr = int(torch.tensor(loadedGC['Nr']).squeeze(0))
num_try = int(torch.tensor(loadedGC['num_try']).squeeze(0))

For linear supply, for each community, we need one more parameter $l$.

In [2]:
'''
l_span = 0.1 + 0.9*torch.rand(R_span.size())
# solve source supply

kappa_span = torch.zeros(R_span.size())

# forlogistic growth, we need to sample g
g_span = 0.1 + 0.9*torch.rand(R_span.size())
# solve carrying capacity
K_span = torch.zeros(R_span.size())
'''
loadlgK = loadmat("./data/LinLogsamples.mat")
l_span = torch.tensor(loadlgK['alll'])
g_span = torch.tensor(loadlgK['allg'])
#kappa_span = torch.tensor(loadlgK['allkappa'])
#K_span = torch.tensor(loadlgK['allK'])

Jnewout = torch.zeros(2,num_try,len(Ns_span),C_span.shape[1]) # 0: linear supply #1: logistic supply

for i in range(len(Ns_span)):
    Ns = int(Ns_span[i]) # different Ns
    for j in range(C_span.shape[1]):
        # different correlations
        C = C_span[i,j,:Ns,:]
        G = G_span[i,j,:Ns,:]
        for k in range(num_try):
            # for different fixed points
            Ss = S_span[i,j,k,:Ns]
            Rs = R_span[i,j,k,:]
            # we can solve the source concentration kappa
            #kappa = (Rs * (Ss @ C))/l_span[i,j,k] + Rs
            #kappa_span[i,j,k] = kappa

            # we can solve the carrying capacity
            #K = (Ss @ C)/g_span[i,j,k] + Rs
            #K_span[i,j,k] = K

            Js = torch.zeros(Ns+Nr,Ns+Nr)
            Js[0:Ns,Ns:Ns+Nr] = torch.diag(Ss) @ G
            Js[Ns:Ns+Nr,0:Ns] = - torch.diag(Rs) @ C.transpose(0,1)
            Js[Ns:Ns+Nr,Ns:Ns+Nr] = - torch.diag(C.transpose(0,1) @ Ss) - torch.diag(l_span[i,j,k])

            E_J = torch.linalg.eigvals(Js).real
            Jnewout[0,k,i,j] = len(E_J[E_J >= 1.0e-6])/Ns # Fraction of Unstable modes of the real Jacobian

            Js[Ns:Ns+Nr,Ns:Ns+Nr] = - torch.diag(g_span[i,j,k] * Rs)
            E_J = torch.linalg.eigvals(Js).real
            Jnewout[1,k,i,j] = len(E_J[E_J >= 1.0e-6])/Ns


In [3]:
from scipy.io import savemat # type: ignore
from pathlib import Path

datapath = Path('./data/').expanduser()
tensorfile = {'Jnewout': Jnewout}
torch.save(tensorfile, datapath/'LinLogJ.pt')

'''
tensordic = {"alll":l_span.numpy(),"allkappa":kappa_span.numpy(),
             "allg":g_span.numpy(),"allK": K_span.numpy()}
savemat("./data/LinLogsamples.mat", tensordic)
'''

'\ntensordic = {"alll":l_span.numpy(),"allkappa":kappa_span.numpy(),\n             "allg":g_span.numpy(),"allK": K_span.numpy()}\nsavemat("./data/LinLogsamples.mat", tensordic)\n'